In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.3.12")
@file:DependsOn("io.ktor:ktor-client-cio:2.3.12")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.3.12")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.3.12")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe


In [21]:
import ro.jf.funds.account.api.model.*
import com.charleskorn.kaml.Yaml
import kotlinx.serialization.builtins.ListSerializer
import kotlinx.serialization.Serializable

val accountYamlFile = File("../../data/provision/accounts.yaml").readText()
val accounts = Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), accountYamlFile)
accounts

[CreateAccountTO(name=Cash RON, unit=Currency(value=RON)), CreateAccountTO(name=Cash EUR, unit=Currency(value=EUR)), CreateAccountTO(name=ING RON, unit=Currency(value=RON)), CreateAccountTO(name=ING Economy, unit=Currency(value=RON)), CreateAccountTO(name=Food Coupons, unit=Currency(value=RON)), CreateAccountTO(name=Revolut RON, unit=Currency(value=RON)), CreateAccountTO(name=Other, unit=Currency(value=RON)), CreateAccountTO(name=BT RON, unit=Currency(value=RON)), CreateAccountTO(name=BT Economy RON, unit=Currency(value=RON))]

In [23]:
import ro.jf.funds.fund.api.model.*

val fundsYamlFile = File("../../data/provision/funds.yaml").readText()
val funds = Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), fundsYamlFile)
funds

[CreateFundTO(name=Expenses), CreateFundTO(name=Work Income), CreateFundTO(name=Savings), CreateFundTO(name=Gifts)]

In [28]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
// TODO(Johann) make this a yaml
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"

val userSdk = UserSdk()
val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

val username = "Johann-13.2"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}
user

UserTO(id=94c0c8a3-2503-4671-a69e-9bcfb109abc0, username=Johann-13.2)

In [26]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = File(ACCOUNTS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), it) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=68147f85-288f-408e-b21b-9bcaec882a7a, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=6a7acecd-0185-4a7b-965c-82c3fd059c7d, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=c3e0c07a-86c1-4b2c-94ef-a951b5b52925, name=ING RON, unit=Currency(value=RON))
AccountTO(id=1500b07e-45e5-4266-bdc9-0103a1beda2e, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=f2be2aa2-abb4-47d4-9127-67fa80ed94da, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=9ee80042-8702-4ae4-af0a-913b4441236d, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=ee45b355-96e2-4933-9c1e-8faf45959c1a, name=Other, unit=Currency(value=RON))
AccountTO(id=113658e5-9aea-44bc-b0b6-f70547b90dbd, name=BT RON, unit=Currency(value=RON))
AccountTO(id=2b5f2c7b-e589-4bd0-8564-41926a7f8a6c, name=BT Economy RON, unit=Currency(value=RON))

In [29]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = File(FUNDS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), it) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=af8eda10-0f16-44d2-b667-1092df4c6a5a, name=Expenses)
FundTO(id=9e2e595f-9643-4bc2-a23e-5023f72672ef, name=Work Income)
FundTO(id=84ced50f-422b-47c4-81d9-d0cc2332f6fa, name=Savings)
FundTO(id=bc67f5e3-b794-4be9-8e02-7bffff9e4553, name=Gifts)

In [30]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawJson: String = File(IMPORT_CONFIGURATION_JSON_FILE).readText()
val jsonFormat = Json { ignoreUnknownKeys = true }
val importConfiguration = jsonFormat.decodeFromString<ImportConfigurationTO>(dataConfigurationRawJson)

In [31]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=b3bf91a1-8078-4595-92ec-83b93fef9dbb, status=COMPLETED, reason=null)